In [1]:
import pandas as pd
import numpy as np
import os
import lightgbm as lgb
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import accuracy_score, f1_score, precision_score, auc, roc_curve, recall_score
from sklearn.metrics import accuracy_score, f1_score, precision_score, auc, roc_curve, recall_score
import warnings 
warnings.filterwarnings("ignore") 
import random
from datetime import timedelta

In [2]:
PARENT_FOLDER = '../data'


kernel_train_path = 'memory_sample_kernel_log_round1_a_train.csv'
failure_train_path = 'memory_sample_failure_tag_round1_a_train.csv'
adress_train_path = 'memory_sample_address_log_round1_a_train.csv'


kernel_test_path = 'memory_sample_kernel_log_round1_b_test.csv'
failure_test_path = 'memory_sample_failure_tag_round1_b_test.csv'
adress_test_path = 'memory_sample_address_log_round1_b_test.csv'

In [3]:
def label(x):
    if x<=60*60*24*7:
        return 1
    else:
        return 0

In [4]:
kernel_var = ['1_hwerr_f', '1_hwerr_e', '2_hwerr_c', '2_sel', '3_hwerr_n', '2_hwerr_s', '3_hwerr_m', '1_hwerr_st',
       '1_hw_mem_c', '3_hwerr_p', '2_hwerr_ce', '3_hwerr_as', '1_ke', '2_hwerr_p', '3_hwerr_kp', '1_hwerr_fl', '3_hwerr_r', '_hwerr_cd',
       '3_sup_mce_note', '3_cmci_sub', '3_cmci_det', '3_hwerr_pi', '3_hwerr_o', '3_hwerr_mce_l']

In [5]:
def etl_kernel(path, agg_time,time):
    data = pd.read_csv(os.path.join(PARENT_FOLDER, path))
    data['collect_time'] = pd.to_datetime(data['collect_time']).dt.floor(agg_time)
    data['count'+time] = 1
    
    if 'tag' in data.columns:
        del data['tag']
        del data['failure_time']
    
    group_data = data.groupby(['serial_number','manufacturer','vendor','collect_time'],as_index=False).agg('sum')

    return group_data

In [6]:
def getLabel(label_path, kernel_df):
    failure_tag = pd.read_csv(os.path.join(PARENT_FOLDER,label_path))
    failure_tag['failure_time']= pd.to_datetime(failure_tag['failure_time'])
    merged_kernel = pd.merge(kernel_df,failure_tag[['serial_number',
                                               'manufacturer','vendor','failure_time']],how='left',on=['serial_number',
                                                                                                       'manufacturer','vendor'])
    merged_kernel['diff_seconds'] = (merged_kernel['failure_time'] - merged_kernel['collect_time']).dt.days*24*60*60 \
                                + ((merged_kernel['failure_time']-merged_kernel['collect_time']).dt.seconds)
    merged_kernel['label'] = merged_kernel['diff_seconds'].map(label)
    return merged_kernel

In [7]:
def sta_repeat(x):
    n = 0
    for i in set(x):
        if x.count(i) > 1:
            n += 1
    return n

In [8]:
def sta_max(x):
    ls = []
    for i in set(x):
        ls.append(x.count(i))
    return max(ls)

In [9]:
def get_Feature(address):
    address['collect_time'] = pd.to_datetime(address['collect_time']).dt.floor('2min')
    address_row = address[['serial_number','collect_time','row']].groupby(['serial_number','collect_time'],as_index=False).agg(list)
    address_col = address[['serial_number','collect_time','col']].groupby(['serial_number','collect_time'],as_index=False).agg(list)
    address_row['row_repeat'] = address_row['row'].map(sta_repeat)
    address_col['col_repeat'] = address_col['col'].map(sta_repeat)
    address_row['row_max'] = address_row['row'].map(sta_max)
    address_col['col_max'] = address_col['col'].map(sta_max)
    address_row['row_num'] = address_row['row'].apply(lambda x: len(set(x)))
    address_col['col_num'] = address_col['col'].apply(lambda x: len(set(x)))
    address_sta = pd.merge(address_col,address_row[['serial_number','collect_time','row_num','row_max','row_repeat']],how='left',on=['serial_number','collect_time'])
    return address_sta

In [10]:
failure_tag = pd.read_csv('../data/memory_sample_failure_tag_round1_a_train.csv')
failure_tag['failure_time']= pd.to_datetime(failure_tag['failure_time'])

In [11]:
failure_data = pd.read_csv('../data/memory_sample_failure_tag_round1_b_test.csv')
failure_data['failure_time'] = pd.to_datetime(failure_data['failure_time'])

In [12]:
address_a = pd.read_csv('../data/memory_sample_address_log_round1_a_train.csv')
address_b = pd.read_csv('../data/memory_sample_address_log_round1_b_test.csv')

In [13]:
address_sta_train = get_Feature(address_a)
address_sta_test = get_Feature(address_b)

In [14]:
address_sta_train.to_csv("address_sta_train.csv")
address_sta_test.to_csv("address_sta_test.csv")